# Notebook 9: Rotations

Rotations are essential in geology. For example, if from a current lineation on a tilted bed we want to estimate the direction of the current that deposited the bed, we need to rotate the bed (and the lineation) back to their pre- tilting orientation.

A rotation is also a coordinate transformation from an old coordinate system to a new coordinate system. The rotation axis is specified by its trend and plunge, and the magnitude of rotation is given by the angle $\omega$ which is positive for clockwise rotation and vice versa. 

Without demonstration, the components of the transformation matrix, $\mathbf{a}$, that describes this transformation are:

$$
\begin{gathered}
a_{11}=\cos \omega+\cos ^2 \alpha(1-\cos \omega) \\
a_{12}=-\cos \gamma \sin \omega+\cos \alpha \cos \beta(1-\cos \omega) \\
a_{13}=\cos \beta \sin \omega+\cos \alpha \cos \gamma(1-\cos \omega) \\
a_{21}=\cos \gamma \sin \omega+\cos \beta \cos \alpha(1-\cos \omega) \\
a_{22}=\cos \omega+\cos ^2 \beta(1-\cos \omega) \\
a_{23}=-\cos \alpha \sin \omega+\cos \beta \cos \gamma(1-\cos \omega) \\
a_{31}=-\cos \beta \sin \omega+\cos \gamma \cos \alpha(1-\cos \omega) \\
a_{32}=\cos \alpha \sin \omega+\cos \gamma \cos \beta(1-\cos \omega) \\
a_{33}=\cos \omega+\cos ^2 \gamma(1-\cos \omega)
\end{gathered}
$$

where $\cos\alpha$, $\cos\beta$ and $\cos\gamma$ are the direction cosines of the rotation axis in our NED coordinate system (see notebook 2).

## Python function

The function below rotates a line (defined by its trend and plunge) about a rotation axis (defined by its trend and plunge), an amount $\omega$.

In [1]:
# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath("../functions"))

In [2]:
# rotate function: this function is also in functions/rotate.py

import numpy as np
from sph_to_cart import sph_to_cart
from cart_to_sph import cart_to_sph

def rotate(rtrd,rplg,rot,trd,plg,ans0):
	"""
	rotate rotates a line by performing a coordinate
	transformation
	
	rtrd = trend of rotation axis
	rplg = plunge of rotation axis
	rot = magnitude of rotation
	trd = trend of the line to be rotated
	plg = plunge of the line to be rotated
	ans0 = A character indicating whether the line 
		to be rotated is an axis (ans0 = "a") or a 
		vector (ans0 = "v")
	trdr and plgr are the trend and plunge of the
		rotated line
	
	NOTE: All angles are in radians
	
	Python function translated from the Matlab function
	Rotate in Allmendinger et al. (2012)
	"""
	# allocate some arrays
	a = np.zeros((3,3)) #Transformation matrix
	raxis = np.zeros(3) #Dir. cosines of rotation axis
	line = np.zeros(3)	#Dir. cosines of line to be rotated
	liner = np.zeros(3) #Dir. cosines of rotated line
	
	# convert rotation axis to direction cosines
	raxis[0] , raxis[1], raxis[2] = sph_to_cart(rtrd,rplg)
	
	# calculate the transformation matrix a for the rotation
	x = 1.0 - np.cos(rot)
	sinrot = np.sin(rot)
	cosrot = np.cos(rot)
	a[0,0] = cosrot + raxis[0]*raxis[0]*x
	a[0,1] = -raxis[2]*sinrot + raxis[0]*raxis[1]*x
	a[0,2] = raxis[1]*sinrot + raxis[0]*raxis[2]*x
	a[1,0] = raxis[2]*sinrot + raxis[1]*raxis[0]*x
	a[1,1] = cosrot + raxis[1]*raxis[1]*x
	a[1,2] = -raxis[0]*sinrot + raxis[1]*raxis[2]*x
	a[2,0] = -raxis[1]*sinrot + raxis[2]*raxis[0]*x
	a[2,1] = raxis[0]*sinrot + raxis[2]*raxis[1]*x
	a[2,2] = cosrot + raxis[2]*raxis[2]*x
	
	# convert trend and plunge of line to be rotated into
	# direction cosines
	line[0] , line[1], line[2] = sph_to_cart(trd,plg)
	
	# perform the coordinate transformation
	for i in range(3):
		for j in range(3):
			liner[i] = a[i,j]*line[j] + liner[i]
	
	# convert to lower hemisphere projection if axis 
	if liner[2] < 0.0 and ans0 == 'a':
		liner *= -1.0
		
	# convert from direction cosines back to trend and plunge
	trdr , plgr = cart_to_sph(liner[0], liner[1], liner[2])
	
	return trdr, plgr

## Application

Let's solve this problem from Leyshon and Lisle (1996): An overturned bed oriented 305/60 (RHR) has sedimentary lineations which indicate the palaeocurrent direction. These pitch at 60NW, with the current flowing up the plunge. Calculate the original trend of the paleocurrents.

To solve this problem, we will do two rotations:
1. A clockwise rotation of the strike line of the bed about the pole of the bed, an amount equal to the pitch of the lineation. This will give us the orientation of the lineation.
2. A clockwise rotation of the lineation about the strike line of the plane, an amount equal to 180 - dip (since the plane is overturned). This will give us the pre-tilted orientation of the lineation.

In [3]:
# import other functions
from pole import pole_from_plane
from zero_twopi import zero_twopi

# strike and dip of the bed in radians
strike, dip = np.radians([305, 60])

# pole to the bed
p_trd, p_plg = pole_from_plane(strike, dip)

# strike line in radians
s_trd, s_plg = strike, 0

# to determine the orientation of the lineation from its pitch
# rotate the strike line clockwise about the pole
# an amount equal to the pitch of the lineation
rot = np.radians(60) # amount of rotation in radians
l_trd, l_plg = rotate(p_trd, p_plg, rot, s_trd, s_plg, 'a')
print(f"Trend/plunge of the lineation:{np.degrees(l_trd):.1f}/{np.degrees(l_plg):.1f}")

# now we need to rotate the lineation about the strike line to its pre-tilted orientation
# the amount of rotation is 180 - dip = 120, and the sense of rotation is clockwise
rot = np.radians(120) # amount of rotation in radians
lr_trd, lr_plg = rotate(s_trd, s_plg, rot, l_trd, l_plg, 'a')
print(f"Pre-tilted trend/plunge of the lineation:{np.degrees(lr_trd):.1f}/{np.degrees(lr_plg):.1f}")

# the current flows up the plunge, so the trend of the paleocurrent is:
pal_trd = zero_twopi(lr_trd + np.pi)
print(f"Trend of the paleocurrent: {np.degrees(pal_trd):.1f}")

Trend/plunge of the lineation:345.9/48.6
Pre-tilted trend/plunge of the lineation:245.0/0.0
Trend of the paleocurrent: 65.0


In the next notebook, we will see that rotations are very handy to draw great and small circles on stereonets.